In [15]:
import numpy as np

from skimage.transform import hough_line, hough_line_peaks
from skimage.feature import canny
from skimage.draw import line as draw_line
from skimage import data
import cv2
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage.transform import probabilistic_hough_line
import random 

# Load image"
# image_path = "/Users/nshelton/Pictures/gundam_title.png"
# image_path = "/Users/nshelton/Pictures/tree.jpg"
image_path = "/Users/nshelton/Pictures/somedragon.jpg"
# image_path = "/Users/nshelton/Pictures/kirby.jpg"


image = cv2.imread(image_path)
# kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
# for i in range(5):
  # image = cv2.erode(image,kernel, cv2.MORPH_CROSS)
# image[:,:,2] = image[:,:,0]

# image_gray = image[:,:,2]


image = cv2.resize(image, (-1,-1), fx = 2, fy = 2)

print(image.shape)

target_color = [0,0,0] 
# target_color = [255,255,255] 
# target_color = [0,255,0] 

mask = image - target_color
mask = np.linalg.norm(mask, axis=2) < 100


color_mask = np.ones((mask.shape[0], mask.shape[1]), np.uint8) * 255
color_mask[mask] = 0

filtered = 255 - color_mask

cv2.imwrite("filtered.png", filtered)

all_strokes_raster = np.ones(filtered.shape) * 255

all_paths = []
 
while filtered.sum() > 0:
    print(filtered.sum(), "\t", len(all_paths),  end="\t\t\t\r")
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(filtered)

    def placeStroke(input_raster, start):
        path = []
        pos = np.array(start)

        while(input_raster[pos[1], pos[0]] > 0):
            path.append(pos.copy())
            pos += [1,1]

        return path
    
    line_thickness = 4
    
    def rasterStroke(dims, path) :
        img = np.zeros(dims)
        for i in range(len(path)-1):
            a = (round(path[i][0]) , round(path[i][1]) )
            b = (round(path[i+1][0]) , round(path[i+1][1]) )
            cv2.line(img, a, b, 0, line_thickness, cv2.LINE_AA)
            
        return img
    
    def rasterStrokeDirect(img, path, color, line_thickness=20) :
        if len(path) == 1:
            a = (round(path[0][0]) , round(path[0][1]) )
            cv2.circle(img, a, line_thickness//2, color, -1)

        for i in range(len(path)-1):
            a = (round(path[i][0]) , round(path[i][1]) )
            b = (round(path[i+1][0]) , round(path[i+1][1]) )
            cv2.line(img, a, b, color, line_thickness, cv2.LINE_4)
    
    new_stroke = placeStroke(filtered, max_loc)
    rasterStrokeDirect(filtered, new_stroke, 0, line_thickness)
    rasterStrokeDirect(all_strokes_raster, new_stroke, 0, line_thickness)

    all_paths.append(new_stroke)

    if (len(all_paths) % 10 == 1):
        cv2.imwrite("current_stroke.png",  (all_strokes_raster).astype(np.uint8))
        cv2.imwrite("filtered.png", filtered)

cv2.imwrite("current_stroke.png",  (all_strokes_raster).astype(np.uint8))




(1664, 1126, 3)


True

In [16]:
import cv2
import numpy as np


from pathUtils import *

RENDER_SCALE = 0.2

# connect_close_paths(path_list)
path_list = [[p.tolist() for p in path] for path in all_paths]
render_img = plotPaths(path_list, RENDER_SCALE)
cv2.imwrite("render.png", render_img)



True

In [17]:
from tqdm import tqdm


import numpy as np
from scipy.spatial import KDTree
 

path_list = [[p.tolist() for p in path] for path in all_paths]

print(len(path_list))


for i in tqdm(range(len(path_list))):
    len_before = len(path_list)
    path_list = merge_one_path_kd(path_list, threshold=5)
    if len_before == len(path_list):
        break

print("merged to", len(path_list))
render_img = plotPaths(path_list, RENDER_SCALE)
cv2.imwrite("render.png", render_img)





11070


 84%|████████▍ | 9323/11070 [04:02<00:45, 38.40it/s]


merged to 1747


True

In [18]:


path_list = close_loops(path_list)

print("closed loops", len(path_list))

path_list  = [np.array(path) for path in path_list]

path_list = smooth_all_2(path_list, 0.2)

print("smoothed 1", len(path_list))

path_list = simplify_all(path_list, 0.001)

print("simplify_all 1", len(path_list))

path_list = smooth_all_2(path_list, 0.01)


print("smoothed 2", len(path_list))

path_list = simplify_all(path_list, 0.5)
print("simplify 2", len(path_list))

render_img = plotPaths(path_list, RENDER_SCALE)
cv2.imwrite("render.png", render_img)


closed loops 1747
smoothed 1 1747
simplify_all 1 1747
smoothed 2 1747
simplify 2 1747


True

In [19]:
 
import json
import uuid

projects_folder = "/Users/nshelton/Hephaestus/projects"

def writePlot(paths):
    dictionary = {
        "created_time": "4/7/2024 4:42:27 PM",
        "modified_time": "4/7/2024 6:27:36 PM",
        "camera_position": [100, 100],
        "zoom": 10.702,
        "aspect": 0.563,
        "dom_element": {},
        "plot_models": [],
    }

    dictionary["plot_models"].append(
        {
            "position": {"x": 0, "y": 0},
            "paths": [path.tolist() for path in path_list],
            "scale": 0.1,
            "id": str(uuid.uuid4()),
            "state": "none",
        }
    )


    with open(projects_folder + "/some_dragon.json", "w") as outfile:
        json.dump(dictionary, outfile)


writePlot(path_list)

